In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dynamixel import AX12
from arm import Arm
import time

In [ ]:
# PORT = '/dev/tty.usbmodem1101'
PORT = '/dev/tty.borgla_test'
BAUD = 1000000
write_packet_delay = 0.5

In [ ]:
# Test connection to all servos by pinging
# We expect error=0 for all 6 servos.  See https://emanual.robotis.com/docs/en/dxl/protocol1/#error
with AX12(PORT, BAUD) as robot:
    robot.ping(0)
    robot.ping(1)
    robot.ping(2)
    robot.ping(3)
    robot.ping(4)
    robot.ping(5)
    # print('test next msg: ', robot.read_next_msg())
    print('test all msgs: ')
    for msg in robot.read_all_msgs():
        print(msg)

In [ ]:
# Write speeds of all servos
with Arm(PORT, BAUD) as robot:
    robot.write_all(AX12.MOVING_SPEED, 25, nbytes=2)

In [ ]:
# Command base servo to 0 degrees
with Arm(PORT, BAUD) as robot:
    robot.command_angle(0, 0)

In [ ]:
# Disable (or enable) all servos
with Arm(PORT, BAUD) as robot:
    robot.write_all(AX12.TORQUE_ENABLE, 0)  # change to 1 to enable

In [ ]:
# Command all servo angles
with Arm(PORT, BAUD) as robot:
    robot.command_angles_deg(0, 0, 0, 0, 0)

In [ ]:
# Record all joint angles
def joint_angles_string(robot):
    joint_angles = robot.read_all_joint_angles_deg()
    return ' '.join([f'{angle:3.0f}' for angle in joint_angles])
with Arm(PORT, BAUD) as robot:
    while True:
        print(joint_angles_string(robot))
        break
        time.sleep(0.5)

In [ ]:
# Paint dipping trajectory
TRAJECTORY_QS_TOLS_TIMEOUTS_PAUSES = [
    ([0, 0, 0, 0, 0], None, None, 0),
    ([0, 50, 110, 0, 90], None, None, 0),
    # ([-8, 53, 130, 0, 61], 2, 3, 1),
    # ([-8, 53, 130, 5, 61], 2, 3, 1),
    # ([-8, 53, 130, -8, 61], 2, 3, 1),
    ([-3, 39, 120, -3, 62], 2, 3, 0),
    ([-3, 64, 133, -3, 73], 2, 3, 0),

    ([0, 50, 90, 0, 90], None, None, 0),
    ([0, 50, 60, 0, 90], None, None, 0),
    ([0, 0, 0, 0, 0], None, None, 0),
]

# Follow trajectory
# THIS IS SUPERCEDED BY arm.py: Arm.execute_joint_path
with Arm(PORT, BAUD) as robot:
    robot.write_all(AX12.TORQUE_ENABLE, 1)
    for i, (q, tol, timeout, pause) in enumerate(TRAJECTORY_QS_TOLS_TIMEOUTS_PAUSES):
        print(f'Going to {q} ({i+1}/{len(TRAJECTORY_QS_TOLS_TIMEOUTS_PAUSES)})')
        # robot.go_to_blocking(q, cb=lambda: print(f'{joint_angles_string(robot):80}', end='\r'))
        print_debug = lambda: print(f'          {joint_angles_string(robot):80}', end='\r')
        if robot.go_to_blocking(q, tol=tol, timeout=timeout, cb=print_debug):
            print("SUCCESS!")
        else:
            print("\nTimeout...  Moving on")
        time.sleep(pause)

In [ ]:
# Painting on window position
REST = [0, 0, 0, 0, 0]
PAINT_ON_WINDOW = [0, 0, -25, 0, -60]
q = REST
# q = PAINT_ON_WINDOW

with Arm(PORT, BAUD) as robot:
    robot.write_all(AX12.TORQUE_ENABLE, 1)

    print_debug = lambda: print(f'          {joint_angles_string(robot):80}', end='\r')
    if robot.go_to_blocking(q, tol=tol, timeout=timeout, cb=print_debug):
        print("SUCCESS!")
    else:
        print("\nTimeout...  Moving on")